<a href="https://colab.research.google.com/github/angui100/Document_Classification_Machine_Learning/blob/master/Invisible_Institute_text_doc_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np

root_path = '/content/drive/My Drive/invisible_institute/CPDP_documents_data_table.csv'

data = pd.read_csv(root_path)

#df = data[pd.notnull(data['text_content'])]
#df.head(10)
data.dropna(subset=['text_content'], inplace=True)
data.tail()

,allegation_id,title,text_content,url,pages,incident_date,most_common_category_id,category,allegation_name,coaccused_count,tag,source_type
1803,1000436,CRID 1000436 CR,REPORT DIGEST LOG NO. TYPE DATE OF REPORT \nCH...,https://assets.documentcloud.org/documents/476...,9,2006-04-21T00:00:00Z,202,Operation/Personnel Violations,Eeo Investigations,2,CR,DOCUMENTCLOUD
1804,1000417,CRID 1000417 CR,COMMAND CHANNEL REVIEW\nCOMPLAINT REGISTER INV...,https://assets.documentcloud.org/documents/457...,70,2006-10-06T00:00:00Z,72,False Arrest,Illegal Arrest / False Arrest,2,CR,DOCUMENTCLOUD
1805,1000301,CRID 1000301 CR,7/18/2018\nView Incident\nPrint Logout Hel...,https://assets.documentcloud.org/documents/476...,23,2006-10-07T00:00:00Z,194,Operation/Personnel Violations,Neglect Of Duty,2,CR,DOCUMENTCLOUD
1806,1000227,CRID 1000227 CR (Discovery),1000227\nCOMMAND CHANNEL REVIEW-SUSTAINED CASE...,https://assets.documentcloud.org/documents/522...,104,2006-05-22T00:00:00Z,199,Operation/Personnel Violations,Misuse Of Department Equipment / Supplies,9,CR,DOCUMENTCLOUD
1807,1000201,CRID 1000201 CR,7/27/2018\nView Incident\nPrint Logout Hel...,https://assets.documentcloud.org/documents/476...,4,2006-10-03T00:00:00Z,109,Use Of Force,Unnecessary Physical Contact / On Duty - No In...,1,CR,DOCUMENTCLOUD


In [3]:
data.tag.unique()

array(['CR', 'CPB'], dtype=object)

In [0]:
data.loc[1807,"text_content"]

'7/27/2018\nView Incident\nPrint   Logout   Help\nView Incident\nHome » Administrative Investigations IAD/COPA » View Incident\nPrint Face Sheet\nPrint Current Case Summary\n< Return\nInvestigation\nReporting Category\nCategory\nSubcategory Situation\nVictim/Offender Weapon\nArmed?\nTypes\nWeapon\nOther\nWeapon\nDeceased?\nRecovered?\n05M UNNECESSARY PHYSICAL CONTACT / ON\nDUTY - NO INJURY\n-\n-\n-\n-\n-\n-\n-\nrow(s) 1 - 1 of 1\nLog No: 1000201 Type:CR\nAccused Members\nReview Name\n-\nVOIGHT,\nWILLIAM\nStatus\nAccused Overall\nFinding\nCLOSED/FINAL UNFOUNDED\nRecommended\nPenalty\nNo. of\nDays\nDate/Time\nServed\n-\n-\n-\nCreated\nDate\n03-OCT2006\nComplimentary\nHistory\nN/A\nrow(s) 1 - 1 of 1\nAttachments\nNo. Type\nRelated No. of\nNarrative\nPerson Pages\nStatus\nApprove Approve\nContent? Inclusion?\n1\nFACE SHEET\nN/A\n-\n-\n-\n-\n2\nCONFLICT CERTIFICATION\nN/A\n-\n-\n-\n-\n3\nCONFLICT CERTIFICATION\nN/A\n-\n-\n-\n-\n4\nRAPID RESPONSE\nINVESTIGATION REPORT\nN/A\n1\n5\nCOMPLAINANT

In [8]:
#data.isna().sum();
#data.describe()
#data.sum()
data.tag.value_counts()

CR     1235
CPB     120
Name: tag, dtype: int64

In [0]:
#reconciled_data = data[["title", 'text_content', 'most_common_category_id', 'category']]
reconciled_data = data[['tag', 'text_content']]
reconciled_data.head()

,tag,text_content
1,CR,SUMMARY REPORT DIGEST-\nCOMPLAINT REGISTER INV...
3,CR,COMMAND CHANNEL REVIEW\nA Sasso\nEmp 14982\nNW...
6,CR,SUMMARY REPORT DIGEST-\nCOMPLAINT REGISTER INV...
8,CR,SUMMARY REPORT DIGEST- DATE OF REPORTIDAY-MO.-...
12,CR,SUM MARY REPORT I DATE OF REPORT IDAY-MO \nCOM...


In [0]:
#df_CPB = data[(data["tag"] == 'CPB')]
#df_CPB.head()

In [0]:
#df_CPB.sum()

In [0]:
#reconciled_data.isna().sum()

In [0]:
#data.isnull().sum()

In [0]:
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re

In [0]:
#Total counts of words in example data
data['text_content'].apply(lambda x: len(x.split(' '))).sum()

3584648

In [0]:
def print_plot(index):
    text = data[data.index == index][['text_content', 'tag']].values[0]
    if len(text) > 0:
        print(text[0])
        print('Tag:', text[1])

In [0]:
print_plot(12)

SUM MARY REPORT I DATE OF REPORT IDAY-MO 
COMPLAINT REGISTER INVESTIGATION NO: _313931 '10 October 2008
CHICAGO POLICE DEPARTMENT 
To be used in all cases that are to be clasSliled as either EXONERATED. UNFOUNDED. NOT SUSTAINED cases where
the 
Inary Recommendation does notexcead FIVE (5) DAYS SUSPENSION.
SUBMIT ORIGINAL AND 3 COPIES IF ASSIGNED TO SAME UNIT AS ACCUSED
SUBMIT ORIGINAL AND 4 COPIES IF NOT ASSIGNED T0 SAME UNIT AS ACCUSED
TO: SUPERINTENDENT OF POLICE
ATTENTION ADMINISTRATOR IN CHARGE. OFFICE OF PROFESSIONAL STANDARDS
ASSISTANT DEPUTY SUPERINTENDENT. INTERNAL AFFAIRS DIVISION
RANK STAR NO. SOCIAL SEC. No .Irnr'II'moN I
VazqueZ. Aurora lnv. 249 1 13 
IIHI DATE I 
01 July 06. 1445 hrs. 222 17
NM ?nmx STAR NU. SOCIAL SEC.NO. 'imir ASSIGI-I 
i Cabrales. Miguel PO 10719 002 
2 
DATE OF APPOINTMENT DUTY STATUS (TIME OF INCIDENTI In] SWORN 133;;
1 14 DEC 98 DUTY OFF DUTY CIVILIAN CODE 01
0 [1 ON DUTY [l DUTY tl swarm 
1? 3- ll CIVILIAN
0W . - CHARGES COURT BRANCH rmTE 
2
a; T- c

In [4]:
nltk.download('stopwords')
#nltk.download('punkt')
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))
#PUNKWORDS = set(punkt.words('english'))
own_set=['_', '#', 'january', 'februay', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december', 'to', 't0', 'of', '0f', 'about'];

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    #text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text    
    text = ' '.join(word for word in text.split() if word not in own_set)
    text = ''.join(word for word in text if word not in own_set)
    text = ''.join([i for i in text if not i.isdigit()]) 
    
    text = re.sub("\s\s+" , " ", text)    
    return text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
#from bs4 import BeautifulSoup

data['text_content'] = data['text_content'].apply(clean_text)

In [8]:
print_plot(12)

sum mary report date report idaymo complaint register investigation chicago police department used cases classliled either exonerated unfounded sustained cases wherethe inary recommendation notexcead five days suspensionsubmit original copies assigned unit accusedsubmit original copies assigned unit accusedto superintendent policeattention administrator charge office professional standardsassistant deputy superintendent internal affairs divisionrank star social sec irnriimon ivazquez aurora lnv iihi date hrs nm nmx star nu social secno imir assigii cabrales miguel po date appointment duty status time incidenti sworn dec duty duty civilian code duty l duty tl swarm civilianw charges court branch rmte a crrr smie telephone corifl o city state telephone sexjrage dob teen rocecity state sexirace conoa edoesee attached sheet adda legationson hours sgt etna lotionhoned ollico promiaiunal registered thiscomplaint behalf nile lisa iousanl emp ii alleged july approximately hours tho location mi

In [0]:
X = data.text_content
y = data.tag
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

## Naive Bayes classifier for multinomial models

In [0]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
import pickle

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)

# save the model to disk
filename = '/content/drive/My Drive/invisible_institute/nb_CPDP_model.sav'
pickle.dump(nb, open(filename, 'wb'))

In [11]:
%%time
my_tags = ['CR', 'CPB']
from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 0.9262899262899262
              precision    recall  f1-score   support

          CR       1.00      0.14      0.25        35
         CPB       0.93      1.00      0.96       372

    accuracy                           0.93       407
   macro avg       0.96      0.57      0.61       407
weighted avg       0.93      0.93      0.90       407

CPU times: user 439 ms, sys: 3.1 ms, total: 442 ms
Wall time: 449 ms


## Logistic regression

In [12]:
from sklearn.linear_model import LogisticRegression
import pickle

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])
logreg.fit(X_train, y_train)

# save the model to disk
filename = '/content/drive/My Drive/invisible_institute/logreg_CPDP_model.sav'
pickle.dump(logreg, open(filename, 'wb'))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [13]:
%%time

y_pred = logreg.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 1.0
              precision    recall  f1-score   support

          CR       1.00      1.00      1.00        35
         CPB       1.00      1.00      1.00       372

    accuracy                           1.00       407
   macro avg       1.00      1.00      1.00       407
weighted avg       1.00      1.00      1.00       407

CPU times: user 435 ms, sys: 423 µs, total: 436 ms
Wall time: 437 ms


## Linear support vector machine

In [0]:
from sklearn.linear_model import SGDClassifier
import pickle

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)

# save the model to disk
filename = '/content/drive/My Drive/invisible_institute/sgd_CPDP_model.sav'
pickle.dump(sgd, open(filename, 'wb'))

In [15]:
%%time

y_pred = sgd.predict(X_test)

print('accuracy %s' % accuracy_score(y_pred, y_test))
print(classification_report(y_test, y_pred,target_names=my_tags))

accuracy 1.0
              precision    recall  f1-score   support

          CR       1.00      1.00      1.00        35
         CPB       1.00      1.00      1.00       372

    accuracy                           1.00       407
   macro avg       1.00      1.00      1.00       407
weighted avg       1.00      1.00      1.00       407

CPU times: user 438 ms, sys: 433 µs, total: 439 ms
Wall time: 440 ms


In [16]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

1.0


## End
